In [1]:
!pip install earthengine-api
from google.colab import auth
auth.authenticate_user()
!earthengine authenticate

E0000 00:00:1764670677.296244    1322 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764670677.303664    1322 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764670677.321839    1322 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764670677.321887    1322 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764670677.321905    1322 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764670677.321910    1322 computation_placer.cc:177] computation placer already registered. Please check linka

In [6]:
import ee
from datetime import datetime
import unicodedata

# ==============================================================================
# 0) AUTH & INIT : Khởi tạo kết nối tới project
# ==============================================================================
try:
    ee.Authenticate()
    ee.Initialize(project='ambient-airlock-472009-u8')
    print("✅ GEE initialized.")
except Exception as e:
    print("❌ Init error:", e)
    raise

# ==============================================================================
# 1) INPUTS: GADM & THỜI GIAN (NĂM 2016). Lấy dữ liệu về ranh giới các vùng huyện của ĐBSCL
# ==============================================================================
admin_level2 = ee.FeatureCollection('projects/ambient-airlock-472009-u8/assets/gadm41_VNM_2')

mekong_provinces = [
    'Long An',
    'Tiền Giang',
    'Bến Tre',
    'Trà Vinh',
    'Vĩnh Long',
    'Đồng Tháp',
    'An Giang',
    'Kiên Giang',
    'Hậu Giang',
    'Sóc Trăng',
    'Bạc Liêu',
    'Cà Mau',
    'Cần Thơ'
]

# Các huyện thuộc các tỉnh ĐBSCL
target_districts = admin_level2.filter(ee.Filter.inList('NAME_1', mekong_provinces))

YEAR = 2016
start = f'{YEAR}-01-01'
end   = f'{YEAR}-12-31'  # FilterDate của GEE là [start, end)

print(f"📅 Running analysis for Year: {YEAR} using ERA5 DAILY Data")

# ==============================================================================
# 2) DỮ LIỆU: ERA5-LAND DAILY AGGREGATED & CROPLAND MASK : Lấy dữ liệu ERA-5 từ ERA-5 Daily
# ==============================================================================
era5_daily = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
               .filterDate(start, end) \
               .filterBounds(target_districts)

# Lấy một ảnh ERA5 bất kỳ để lấy projection/scale chuẩn
sample_era5 = ee.Image(era5_daily.first())
era5_proj = sample_era5.projection()

# --- TẠO CROPLAND MASK TỪ DYNAMIC WORLD V1 (theo NĂM, KHÔNG DÙNG FRACTION THRESHOLD) ---

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
           .filterDate(f'{YEAR}-01-01', f'{YEAR}-12-31') \
           .filterBounds(target_districts)

dw_sample = ee.Image(dw_col.first())
dw_proj = dw_sample.select('label').projection()

# 1) Lấy MODE theo thời gian cho band 'label'
dw_label_mode = dw_col.select('label').reduce(ee.Reducer.mode())

# 2) Cropland 10m nhị phân (0/1): class 4 = crops
dw_crops_10m = dw_label_mode.eq(4).clip(target_districts)

# 3) “Hạ” ảnh DW xuống scale 100m để giảm số pixel input khi gom lên ERA5
dw_proj_100m = dw_proj.atScale(100)

dw_crops_100m = dw_crops_10m.reproject(dw_proj_100m) \
                             .setDefaultProjection(dw_proj_100m)

# 4) GOM 100m -> ~11km (ERA5) VỚI max():
#    - max(0/1) = 1 nếu trong pixel ERA5 có ÍT NHẤT 1 pixel cropland
#    - tương đương: "chỉ cần có cropland là giữ", không dùng fraction, không threshold
cropland_any_era5 = dw_crops_100m.reduceResolution(
    reducer=ee.Reducer.max(),
    maxPixels=65535
).reproject(era5_proj)

# 5) Mask cropland nhị phân ở scale ERA5
cropland_mask = cropland_any_era5.eq(1)

# ==============================================================================
# 3) HÀM XỬ LÝ CHO TỪNG ẢNH (Mỗi ảnh là 1 ngày)
# ==============================================================================
def process_day(image):
    img = image.reproject(era5_proj)

    # --- A. CHUYỂN ĐỔI ĐƠN VỊ & ĐỔI TÊN ---
    temp = img.select('temperature_2m').subtract(273.15).rename('2m_temperature_mean')
    precip   = img.select('total_precipitation_sum').multiply(1000).rename('total_precipitation_sum')
    evap     = img.select('total_evaporation_sum').multiply(1000).rename('total_evaporation_sum')
    pot_evap = img.select('potential_evaporation_sum').multiply(1000).rename('potential_evaporation_sum')

    other_means = img.select(
        ['volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2',
         'surface_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m']
    ).rename([
        'volumetric_soil_water_layer_1_mean', 'volumetric_soil_water_layer_2_mean',
        'surface_pressure_mean', '10m_u_component_of_wind_mean', '10m_v_component_of_wind_mean'
    ])

    other_sums = img.select(
        ['surface_net_solar_radiation_sum', 'surface_solar_radiation_downwards_sum']
    )

    final_img = ee.Image([temp, precip, evap, pot_evap, other_means, other_sums])

    # --- B. MASK CROPLAND Ở SCALE ERA5 (theo cropland_any_era5) ---
    final_img_masked = final_img.updateMask(cropland_mask)

    # --- C. REDUCE REGIONS ---
    date_str = img.date().format('YYYY-MM-dd')

    stats = final_img_masked.reduceRegions(
        collection=target_districts,
        reducer=ee.Reducer.mean(),
        scale= sample_era5.projection().nominalScale(),
        tileScale=4
    ).map(lambda f: f.set('date', date_str))

    return stats

# ==============================================================================
# 4) CHẠY MAP & FLATTEN
# ==============================================================================
print("⏳ Mapping over collection...")

final_fc = era5_daily.map(process_day).flatten()

# Lọc bỏ record không có giá trị (huyện/ngày không có pixel cropland nào)
final_fc = final_fc.filter(ee.Filter.notNull(['2m_temperature_mean']))

# ==============================================================================
# 5) EXPORT
# ==============================================================================
output_cols = [
    '2m_temperature_mean',
    'total_precipitation_sum',
    'total_evaporation_sum',
    'potential_evaporation_sum',
    'volumetric_soil_water_layer_1_mean',
    'volumetric_soil_water_layer_2_mean',
    'surface_pressure_mean',
    '10m_u_component_of_wind_mean',
    '10m_v_component_of_wind_mean',
    'surface_net_solar_radiation_sum',
    'surface_solar_radiation_downwards_sum'
]
id_cols = ['NAME_1', 'GID_1', 'NAME_2', 'GID_2', 'date']

selectors = id_cols + output_cols

# Normalize the province name to remove diacritics and replace spaces with underscores
normalized_province_name = unicodedata.normalize('NFKD', mekong_provinces[0]).encode('ascii', 'ignore').decode('utf-8').replace(' ', '_')
output_filename = f'{YEAR}_ERA5Land_DailyAggr_MekongDelta_DW_CroplandAnyMask'

task = ee.batch.Export.table.toDrive(
    collection=final_fc,
    description=output_filename,
    folder='GEE_ERA5_Data_Exports',
    fileNamePrefix=output_filename,
    fileFormat='CSV',
    selectors=selectors
)

task.start()
print(f"🚀 Export started: {output_filename}")


✅ GEE initialized.
📅 Running analysis for Year: 2016 using ERA5 DAILY Data
⏳ Mapping over collection...
🚀 Export started: 2016_ERA5Land_DailyAggr_MekongDelta_DW_CroplandAnyMask
